# 110. Basic Linear Regression

# 1) Feed Forward 구현

- weight와 bias를 알고 있다고 가정하고 Feed Forward 함수를 직접 구현  
- 다음의 선형회귀 문제를 pytorch 로 해결  

$$b=-1,w=2$$

$$\hat{y}=-1+2x$$

In [1]:
import numpy as np
import torch

w = torch.tensor(2.0)
b = torch.tensor(-1.0)
w, b

(tensor(2.), tensor(-1.))

In [2]:
def forward(x):
    yhat = w * x + b
    return yhat

x = 1 일 때 `y = 2x - 1`이므로 y 는 1

In [4]:
x = torch.tensor([1.0])
yhat = forward(x)
yhat.item()

1.0

x 가 multiple inputs (array) 인 경우:

In [5]:
x = torch.tensor([[1.0], [2.0], [3.0]])
forward(x)

tensor([[1.],
        [3.],
        [5.]])

<h1>2) torch.nn 의 Linear 클래스를 이용한 Linear Regression</h1>

In [6]:
from torch.nn import Linear
torch.manual_seed(101)

model = Linear(in_features=1, out_features=1, bias=True)

<code>state_dict()</code> method 는 각 layer 의 parameter tensor 를 dictionary 로 반환하므로 직접 값을 update 할 수 있다.

In [11]:
model.state_dict()['bias'], model.weight, model.bias

(tensor([-0.0994]),
 Parameter containing:
 tensor([[-0.6039]], requires_grad=True),
 Parameter containing:
 tensor([-0.0994], requires_grad=True))

In [12]:
model.state_dict()['weight'][0] = torch.tensor(2)
model.state_dict()['bias'][0] = torch.tensor(-1)
model.state_dict()

OrderedDict([('weight', tensor([[2.]])), ('bias', tensor([-1.]))])

In [17]:
x = torch.tensor([[1.0]])
yhat = model(x)
yhat.item()

1.0

In [20]:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)
yhat.detach().numpy()

array([[1.],
       [3.],
       [5.]], dtype=float32)

<h1>3) 사용자 정의 Module 작성</h1>

In [21]:
from torch import nn

class LR(nn.Module):
    def __init__(self, input_size, output_size):
        super(LR, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [22]:
model = LR(1, 1)
model.state_dict()

OrderedDict([('linear.weight', tensor([[-0.8182]])),
             ('linear.bias', tensor([0.7744]))])

In [24]:
model.state_dict()['linear.weight'][0] = torch.tensor(2)
model.state_dict()['linear.bias'][0] = torch.tensor(-1)
model.state_dict()

OrderedDict([('linear.weight', tensor([[2.]])),
             ('linear.bias', tensor([-1.]))])

In [28]:
print(list(model.parameters()))
print(model.linear)

[Parameter containing:
tensor([[2.]], requires_grad=True), Parameter containing:
tensor([-1.], requires_grad=True)]
Linear(in_features=1, out_features=1, bias=True)


In [30]:
x = torch.tensor([[1.0], [2.0], [3.0]])
yhat = model(x)
yhat.detach().numpy()

array([[1.],
       [3.],
       [5.]], dtype=float32)

# Backpropagation 과 경사하강법 적용

- weight 와 bias를 스스로 학습
- (온도, 강수량, 습도)를 feature 로 입력 받아  (사과, 오렌지)의 수확량(label)을 선형회귀로 예측하는 model 작성

## Data

In [33]:
# Input (습도, 강수량, 최고온도, 최저온도)
inputs = np.array([
                   [73, 67, 43, 10], 
                   [91, 88, 64, 5], 
                   [87, 134, 58, 2], 
                   [102, 43, 37, 4], 
                   [69, 96, 70, 5]], dtype='float32')

In [34]:
# Targets - (apples, oranges) 수확량
targets = np.array([
                    [56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

- input 과 target을 tensor로 변환

In [35]:
inputs = torch.from_numpy(inputs)
inputs

tensor([[ 73.,  67.,  43.,  10.],
        [ 91.,  88.,  64.,   5.],
        [ 87., 134.,  58.,   2.],
        [102.,  43.,  37.,   4.],
        [ 69.,  96.,  70.,   5.]])

In [36]:
targets = torch.from_numpy(targets)
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [37]:
inputs.size(), targets.size()

(torch.Size([5, 4]), torch.Size([5, 2]))

# 1) torch.nn Linear Class 직접 사용

In [38]:
from torch.nn import Linear
from torch import optim
torch.manual_seed(101)

In [40]:
model = Linear(in_features=4, out_features=2)

In [42]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-4)

In [43]:
for epoch in range(100):
    for x, y in zip(inputs, targets):
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [44]:
preds = model(inputs)
loss = criterion(preds, targets)
loss

tensor(1.0233, grad_fn=<MseLossBackward0>)

In [45]:
targets.numpy()

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)

In [48]:
preds.detach().numpy()

array([[ 58.09486 ,  69.60697 ],
       [ 82.23586 , 100.481476],
       [117.720436, 132.58585 ],
       [ 20.790157,  36.927452],
       [102.21539 , 118.98051 ]], dtype=float32)

# 2) 사용자 정의 model 사용

- Linear Regression 사용자 정의 model 

In [55]:
class LinearReg(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearReg, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        yhat = self.fc(x)
        return yhat

In [56]:
inputs.shape, targets.shape

(torch.Size([5, 4]), torch.Size([5, 2]))

In [57]:
model = LinearReg(inputs.shape[1], targets.shape[1])

In [58]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-4)

model.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.0224, -0.4284,  0.0834,  0.2521],
                      [ 0.2649, -0.3557,  0.2152, -0.1047]])),
             ('fc.bias', tensor([ 0.1244, -0.1316]))])

In [59]:
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.0001,
   'momentum': 0,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'maximize': False,
   'foreach': None,
   'params': [0, 1]}]}

## Train the model

In [60]:
for epoch in range(100):
    for x, y in zip(inputs, targets):
        yhat = model(x)
        loss = criterion(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [61]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4054,  0.8456,  0.6950,  0.0985],
         [-0.2953,  0.7928,  0.9169, -0.1237]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1192, -0.1316], requires_grad=True)]

In [62]:
model.state_dict()

OrderedDict([('fc.weight',
              tensor([[-0.4054,  0.8456,  0.6950,  0.0985],
                      [-0.2953,  0.7928,  0.9169, -0.1237]])),
             ('fc.bias', tensor([ 0.1192, -0.1316]))])

In [63]:
preds = model(inputs)
preds

tensor([[ 58.0502,  69.6239],
        [ 82.6131, 100.8325],
        [118.6688, 133.3539],
        [ 21.2376,  37.2740],
        [102.4668, 119.1725]], grad_fn=<AddmmBackward0>)

In [66]:
targets.numpy()

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)

In [68]:
preds.detach().numpy()

array([[ 58.050175,  69.623886],
       [ 82.613075, 100.8325  ],
       [118.66878 , 133.35387 ],
       [ 21.237638,  37.274048],
       [102.46683 , 119.17251 ]], dtype=float32)

## Save and Load Model

In [70]:
torch.save(model.state_dict(), 'model.pt')

- model reload 및 parameter 복원 확인

In [ ]:
reloaded = Linr